In [0]:
#@title mount drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#@title imports
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.layers import Input, Dense,Concatenate, Flatten, Reshape, Permute, BatchNormalization, Dropout, Activation, InputLayer, Conv2DTranspose
from keras.optimizers import RMSprop
from keras.models import model_from_json, Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.layers.advanced_activations import ELU
from keras.layers import Lambda
from keras.losses import binary_crossentropy, sparse_categorical_crossentropy
from keras.layers.merge import concatenate
import numpy as np
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import precision_recall_curve
import keras

Using TensorFlow backend.


In [0]:
#@title compare true faults images with predicted
import pylab as plt

def _check_isnan(
    image, 
    map_true,
    
    map_pred,
    titles,
):
    if np.all(np.isnan(image)):
        image = np.zeros(image.shape)
        
    if np.all(np.isnan(map_true)):
        map_true = np.zeros(image.shape)
        
    if np.all(np.isnan(map_pred)):
        map_pred = np.zeros(image.shape)
        
    titles = np.squeeze(np.array(titles))
    
    if not ((len(titles.shape)==1) & (len(titles) >= image.shape[0])):
        
        titles = np.arange(image.shape[0])
    
    return image, map_true, map_pred, titles

def _check_shape(
    image, 
    map_true,
    map_pred,
):
    if not image.shape == map_true.shape:
        raise Exception(
            'image.shape must be equal map_true.shape.'
            'The shape of image was: {}.'
            'The shape of map_true was: {}'.format(image.shape, map_true.shape)
        )
        
    if not image.shape == map_pred.shape:
        raise Exception(
            'image.shape must be equal map_pred.shape. '
            'The shape of image was: {}.'
            ' The shape of map_pred was: {}'.format(image.shape, map_pred.shape)
        )


def _check_input(
    image, 
    map_true,
    map_pred,
    titles,
):
    if len(image.shape) < 3:
        image = image[None, ...].copy()
    
    image, map_true, map_pred, titles = _check_isnan(image, map_true, map_pred, titles)
    
    if len(map_true.shape) < 3:
        map_true = map_true[None, ...].copy()
    
    if len(map_pred.shape) < 3:
        map_pred = map_pred[None, ...].copy()
        
    _check_shape(image, map_true, map_pred)
    
    return image, map_true, map_pred, titles


def show_image_dashboard(
    image, 
    map_true=np.nan, 
    map_pred=np.nan, 
    threshold=.5, 
    alpha=.5, 
    titles=np.nan, 
    scale_map=True,
    fig_hight=3,
    n_cols=3,
    image_cmap='seismic',
    map_true_cmap='Greys_r',
    map_pred_cmap='Greens',
    axis_off=True,
):
    
    title_format = '{}'
    image, map_true, map_pred, titles = _check_input(image, map_true, map_pred, titles)
    
    n_rows = image.shape[0] / n_cols
    n_rows = np.int32(np.floor(n_rows)) + 1
    
    
    fig_width = fig_hight * int(image.shape[2] / image.shape[1]) * n_cols
    
    j_zx = 0
    for jr in range(n_rows):
        _image = image[jr*n_cols: (jr+1)*n_cols].copy()
        _map_true = map_true[jr*n_cols: (jr+1)*n_cols].copy()
        _map_pred = map_pred[jr*n_cols: (jr+1)*n_cols].copy()
        
        _, axs = plt.subplots(figsize=(fig_width, fig_hight), ncols=n_cols, nrows=1)
        for i in range(n_cols):
            j_img = jr*n_cols + i
            if j_img >= image.shape[0]:
                continue
            cur_image = image[j_img].copy()
            cur_map_true = map_true[j_img].copy()
            cur_map_pred = map_pred[j_img].copy()
            
            cur_map_true = np.float32(cur_map_true)
            cur_map_pred = np.float32(cur_map_pred)
            
            if scale_map:
                cur_map_pred /=  np.abs(cur_map_pred).max() + 1e-16
            
            
            cur_map_pred[cur_map_pred < threshold] = np.nan
            cur_map_true[cur_map_true < threshold] = np.nan

            axs[i].imshow(cur_image, cmap=image_cmap, aspect='equal')

            if not np.all(np.isnan(cur_map_pred)):
                axs[i].imshow(cur_map_pred, alpha=alpha, cmap=map_pred_cmap, aspect='equal')

            if not np.all(np.isnan(cur_map_true)):
                #pass
                axs[i].imshow(cur_map_true, alpha=alpha, cmap=map_true_cmap, aspect='equal')

            axs[i].set_title(title_format.format(titles[j_img]))
            if axis_off:
                axs[i].set_axis_off()
            j_zx += 1
        [axs[i].set_axis_off() for i in range(n_cols) if i >=_image.shape[0]]
        plt.show()

In [0]:
#@title model callback
import keras
from IPython.display import clear_output
import pylab as plt

from itertools import product


class PlotLosses(keras.callbacks.Callback):
  
  
    def __init__(self):
      self.epoch_losses = {'ssim':[], 'precision':[], 'recall':[], 'fbeta_score':[],'iou':[]}
      self.m_names = ['ssim', 'precision', 'recall', 'fbeta_score','iou']
      self.e = 0
      #0a0b0c
      
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()

        self.logs = []
        
    def on_train_end(self, logs={}):
        fig, ax = plt.subplots()
        x = np.arange(1, self.e + 1)
        plt.title("epoch: {}".format(self.e + 1))
        
        for m in self.m_names:
            ax.plot(x, self.epoch_losses[m], label=m)
        axes = plt.gca()
        axes.set_ylim([0,1])
        plt.legend()
        plt.show()
    

    def on_epoch_end(self, epoch, logs={}):
        #if not logs:
          #return
        self.e = epoch
        self.logs.append(logs)
        self.x.append(self.i)
        
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        if self.i < self.params['epochs'] :
            clear_output(wait=True)
            l = round(float(logs.get('loss')), 4)
            v_l = round(float(logs.get('val_loss')), 4)
            #ssim = round(float(logs.get('ssim')), 4)
            
            m_values =[]
            
            for m in  self.m_names:
                v = float(logs.get(m))
                self.epoch_losses.get(m).append(v) 
                m_values.append(v)
            
            fig = plt.figure(figsize=(34, 20))
            loss_plot = fig.add_subplot(4, 4, 1)
            loss_plot.plot(self.x, self.losses, label="loss")
            loss_plot.plot(self.x, self.val_losses, label="v_loss")
            loss_plot.legend()
            
            p = fig.add_subplot(4,4,2)
        
            
            x = np.arange(len(self.m_names))
            p.bar(x, m_values)
            axes = plt.gca()
            axes.set_ylim([0,1])
            plt.xticks(x, self.m_names)
            
            plt.legend()
            plt.show()
            
#l = PlotLosses()
#l.on_epoch_end(1)


In [0]:
#@title custom loss
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = keras.backend.variable(weights)

    def loss(y_true, y_pred):
        #print(y_true)
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= keras.backend.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = keras.backend.clip(y_pred, keras.backend.epsilon(), 1 - keras.backend.epsilon())
        # calc
        loss = y_true * keras.backend.log(y_pred) * weights
        loss = -keras.backend.sum(loss, -1)
        return loss

    return loss

In [0]:
#@title Metrics


    
def ssim(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, 1.)
  
def iou(true, pred):

    intersection = true * pred

    notTrue = 1 - true
    union = true + (notTrue * pred)

    return K.sum(intersection)/K.sum(union)
  
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    y_true, y_pred = y_true[...,-1], y_pred[...,-1]
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    y_true, y_pred = y_true[...,-1], y_pred[...,-1]
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    y_true, y_pred = y_true[...,-1], y_pred[...,-1]
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score
  
  
METRICS = ['accuracy',
           ssim, 
           precision, 
           recall, 
           fbeta_score,
           iou,
          ]

In [0]:
#@title Load Lapteva_faults data
import pickle
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

with open('/content/drive/My Drive/Seismic/Lapteva_faults_10k.pickle', 'rb') as handle:
    data_set = pickle.load(handle)  

X_set = data_set['X_set'] 
y_set = data_set['y_set'] 
Titles = data_set['fault_title'] 
#print(np.roll(y_set, -1, axis=2))

X_train, X_test, y_train, y_test = train_test_split(
    X_set, y_set, test_size=.20, random_state=35
)


#Для unet прогноз подсвета 
#y_train = y_train[..., None]
#y_test = y_test[..., None]

# в каждом пикселе прогнозируем вероятность разлома и не разлома
y_train = to_categorical(y_train, num_classes=2) 
y_test = to_categorical(y_test, num_classes=2)


In [0]:
#@title Unet model
from keras.layers.merge import concatenate, add

#@title Unet Sesm
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x
  
def get_unet_arc(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    #outputs = Conv2D(2, (1, 1), activation='softmax') (c9)
    
    h = Dense(8, activation='relu')(c9)

    outputs = Dense(2, activation='softmax')(h)
    
    model = Model(inputs=[input_img], outputs=[outputs])
    return model
  
  
def create_unet():  
    input_img = Input((64, 64, 1), name='img')
    model = get_unet_arc(input_img, n_filters=16, dropout=0.05, batchnorm=True)
    
    
    optimizer = keras.optimizers.Adam(
        lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False
    )
    
    loss = weighted_categorical_crossentropy(np.array([.1, 1]))
    
    
    model.compile(optimizer='adam', loss=loss, metrics=METRICS)
    #model.summary()
    return model

#create_unet_sesm()



In [0]:
#@title my unet
#@title Unet model
from keras.layers.merge import concatenate, add

#@title Unet Sesm
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, 
               kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x
  
def get_unet_arc(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    #outputs = Conv2D(2, (1, 1), activation='softmax') (c9)
    
    h = Dense(8, activation='relu')(c9)

    outputs = Dense(2, activation='softmax')(h)
    
    model = Model(inputs=[input_img], outputs=[outputs])
    return model
  
  
def create_my_unet():  
    input_img = Input((64, 64, 1), name='img')
    model = get_unet_arc(input_img, n_filters=16, dropout=0.05, batchnorm=True)
    
    
    optimizer = keras.optimizers.Adam(
        lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False
    )
    
    loss = weighted_categorical_crossentropy(np.array([.1, 1]))
    
    
    model.compile(optimizer='adam', loss=loss, metrics=METRICS)
    #model.summary()
    return model

#create_unet_sesm()



In [0]:
#@title fit


model = create_unet()
pltclb = PlotLosses()
#print(model.summary())


model.fit(
    X_train[...,None],
    y_train,
    epochs=64,
    batch_size=64,
    shuffle=True,
    callbacks=[pltclb],
    validation_split= .1,
    verbose=1,
    validation_steps=None
)

In [0]:
#@title evaluate
l = model.evaluate(X_test[..., None], y_test, batch_size=64, verbose=0)

for i,n in enumerate(model.metrics_names):
    print(n + ": " + str(l[i]))

plt.subplot(111)
x = np.arange(len(model.metrics_names))
plt.bar(x, l)
axes = plt.gca()
axes.set_ylim([0,1])
plt.xticks(x, model.metrics_names)
plt.legend()
plt.show()


In [0]:
#@title Rock (precision recall)

# добавили 1 размерность в конце

X = pred.reshape(pred.shape + (1,))
y = y_test.reshape(y_test.shape + (1,))

y_score = X[:, :, :, 1:]

#фильтр по y где только разлом
y_label = y[:, :, :, 1:] > 0.5

y_score = y_score.reshape([y_score.size,])
y_label = y_label.reshape([y_label.size,])

precision, recall, _ = precision_recall_curve(y_label, y_score)

step_kwargs = ({'step': 'post'})
               #if 'step' in signature(plt.fill_between).parameters
               #else {})

plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

In [0]:
#@title show images

pred = model.predict(X_test[..., None])
print(pred.shape)
sl_s = 120
sl_e = 150
show_image_dashboard(X_test[sl_s:sl_e], y_test[...,1][sl_s:sl_e],
     pred[...,1][sl_s:sl_e], titles=Titles, fig_hight=4, threshold=.5, alpha=None, n_cols=6)